## Welcome to the Second Lab - Week 1, Day 3

Today we will work with lots of models! This is a way to get comfortable with APIs.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Important point - please read</h2>
            <span style="color:#ff7800;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you carefully execute this yourself, <b>after</b> watching the lecture. Add print statements to understand what's going on, and then come up with your own variations.<br/><br/>If you have time, I'd love it if you submit a PR for changes in the community_contributions folder - instructions in the resources. Also, if you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...
            </span>
        </td>
    </tr>
</table>

In [1]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import json
import concurrent.futures
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display


In [2]:
# Always remember to do this!
load_dotenv(override=True)

True

In [5]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_


In [6]:
request = "Please come up with a challenging, nuanced short question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [7]:
messages

[{'role': 'user',
  'content': 'Please come up with a challenging, nuanced short question that I can ask a number of LLMs to evaluate their intelligence. Answer only with the question, no explanation.'}]

In [7]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-5-mini",
    messages=messages,
)
question = response.choices[0].message.content
print(question)


How would you prove to a skeptical alien that you truly understand the concept of "red" without using human language or showing visual examples?


In [8]:
competitors = []
answers = []
messages = [{"role": "user", "content": question}]

## Note - update since the videos

I've updated the model names to use the latest models below, like GPT 5 and Claude Sonnet 4.5. It's worth noting that these models can be quite slow - like 1-2 minutes - but they do a great job! Feel free to switch them for faster models if you'd prefer, like the ones I use in the video.

## Note - Skip these following steps if you want to execute your (Ehsan's concurrent execution of the models, which enables parallelization) [Jump to Exercise](#ehsan-exercise)

In [ ]:
# The API we know well
# I've updated this with the latest model, but it can take some time because it likes to think!
# Replace the model with gpt-4.1-mini if you'd prefer not to wait 1-2 mins

model_name = "gpt-5-nano"

response = openai.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
# Anthropic has a slightly different API, and Max Tokens is required

model_name = "claude-sonnet-4-5"

claude = Anthropic()
response = claude.messages.create(model=model_name, messages=messages, max_tokens=1000)
answer = response.content[0].text

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
# Note: How OpenAI SDK compatibility with Gemini works:
# - SDK: You use the standard openai Python SDK (OpenAI library).
# - Endpoint: Set base_url to https://generativelanguage.googleapis.com/v1beta/openai/ to redirect requests to Gemini's servers.
# - API Key: Use your Gemini API Key from Google AI Studio (not an OpenAI key).
# This compatibility is intentional, allowing developers to test or switch to Gemini models with minimal code changes, using the same OpenAI-based codebase.

gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
#Deep-Seek
deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
model_name = "deepseek-chat"

response = deepseek.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
# Groq
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "openai/gpt-oss-120b"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)


## For the next cell, we will use Ollama

Ollama runs a local web service that gives an OpenAI compatible endpoint,  
and runs models locally using high performance C++ code.

If you don't have Ollama, install it here by visiting https://ollama.com then pressing Download and following the instructions.

After it's installed, you should be able to visit here: http://localhost:11434 and see the message "Ollama is running"

You might need to restart Cursor (and maybe reboot). Then open a Terminal (control+\`) and run `ollama serve`

Useful Ollama commands (run these in the terminal, or with an exclamation mark in this notebook):

`ollama pull <model_name>` downloads a model locally  
`ollama ls` lists all the models you've downloaded  
`ollama rm <model_name>` deletes the specified model from your downloads

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Super important - ignore me at your peril!</h2>
            <span style="color:#ff7800;">The model called <b>llama3.3</b> is FAR too large for home computers - it's not intended for personal computing and will consume all your resources! Stick with the nicely sized <b>llama3.2</b> or <b>llama3.2:1b</b> and if you want larger, try llama3.1 or smaller variants of Qwen, Gemma, Phi or DeepSeek. See the <A href="https://ollama.com/models">the Ollama models page</a> for a full list of models and sizes.
            </span>
        </td>
    </tr>
</table>

In [ ]:
!ollama pull llama3.2

In [ ]:
ollama = OpenAI(base_url='http://99.251.9.3:11434/v1', api_key='ollama')
model_name = "llama3.2"

response = ollama.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
# So where are we?

print(competitors)
# print(answers)


In [ ]:
# It's nice to know how to use "zip"
for competitor, answer in zip(competitors, answers):
    print(f"Competitor: {competitor}\n\n{answer}")


In [ ]:
# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [ ]:
print(together)

In [ ]:
judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

```
{together}
```

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [ ]:
print(judge)

In [ ]:
judge_messages = [{"role": "user", "content": judge}]

In [ ]:
# Judgement time!

openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-5-mini",
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)


In [ ]:
# OK let's turn this into results!
results_dict = json.loads(results)
ranks = results_dict["results"]
print(ranks)
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")
    
    
# Ehsan added this to make it a oneliner.    
# [ WHAT_I_WANT_TO_SAVE ] for [ TEMPORARY_VARIABLE_NAME ] in [ THE_LIST ]
ranked = {f"Rank {idx+1}":competitors[int(rank)-1] for idx, rank in enumerate(ranks)}
ranked = json.dumps(ranked)
print(ranked)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Which pattern(s) did this use? Try updating this to add another Agentic design pattern.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">These kinds of patterns - to send a task to multiple models, and evaluate results,
            are common where you need to improve the quality of your LLM response. This approach can be universally applied
            to business projects where accuracy is critical.
            </span>
        </td>
    </tr>
</table>

## Ehsan Exercise

In [ ]:
# Ehsan: I am going to follow the parallelization path of the excersize
# For that, lets create new instances of each LLM class:
from email import message


openai_client = OpenAI()
openai_model = "gpt-5-nano"

claude_client = Anthropic()
claude_model="claude-sonnet-4-5"

gemini_client = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
gemini_model = "gemini-2.5-flash"

deepseek_client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
deepseek_model = "deepseek-chat"

groq_client = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
groq_model = "openai/gpt-oss-120b"

ollama_client = OpenAI(api_key='ollama', base_url="http://99.251.9.3:11434/v1")
ollama_model = "llama3.2"

# Now create functions to wrap every LLM call
# Note: Anthropic uses a different API, so it needs its own function
def call_openai_compatible(client, messages, model_name: str) -> str:
    """For OpenAI-compatible APIs (OpenAI, Gemini, DeepSeek, Groq)"""
    response = client.chat.completions.create(model=model_name, messages=messages)
    return response.choices[0].message.content

def call_anthropic(client, messages, model_name: str) -> str:
    """For Anthropic's API"""
    response = client.messages.create(model=model_name, messages=messages, max_tokens=1000)
    return response.content[0].text

# Now run them in parallel!
# Note: We pass the function reference WITHOUT calling it (no parentheses after the function name)
# executor.submit() will call it for us in a separate thread
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {
        "openai": executor.submit(call_openai_compatible, openai_client, messages, openai_model),
        "anthropic": executor.submit(call_anthropic, claude_client, messages, claude_model),
        "gemini": executor.submit(call_openai_compatible, gemini_client, messages, gemini_model),
        "deepseek": executor.submit(call_openai_compatible, deepseek_client, messages, deepseek_model),
        "groq": executor.submit(call_openai_compatible, groq_client, messages, groq_model),
        "ollama": executor.submit(call_openai_compatible, ollama_client, messages ,ollama_model)
    }
    
    # Wait for all to complete and collect results
    # f.result() returns the string from each LLM
    # enumerate(futures.items()) returns (index, (name, future))
    # So we unpack as: (index, (name, f))
    # Store as list of dicts with index, name, and result
    results_with_names = [{"name": name, "result": f.result()} for (name, f) in futures.items()]
    # The following line is also works:
    # results_with_names = [{"name": name, "result": futures[name].result()} for name in futures]
    # Note: Must unpack as (index, (name, f)) because enumerate(futures.items()) returns (index, (name, future))
    
    # Use _ to ignore the name if you don't need it
    results_with_no_name = [{"result": f.result()} for (_, f) in futures.items()]
    #This is also possible
    # results_with_no_name = [{"result": futures[name].result()} for name in futures] 
    

print(results_with_no_name);



In [9]:
#Here we are doing the same thing we did in the previous rung but using asyncio
import asyncio
from email import message


openai_client = OpenAI()
openai_model = "gpt-5-nano"

claude_client = Anthropic()
claude_model="claude-sonnet-4-5"

gemini_client = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
gemini_model = "gemini-2.5-flash"

deepseek_client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
deepseek_model = "deepseek-chat"

groq_client = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
groq_model = "openai/gpt-oss-120b"

ollama_client = OpenAI(api_key='ollama', base_url="http://99.251.9.3:11434/v1")
ollama_model = "llama3.2"

# Now create functions to wrap every LLM call
# Note: Anthropic uses a different API, so it needs its own function
def call_openai_compatible(client, messages, model_name: str) -> str:
    """For OpenAI-compatible APIs (OpenAI, Gemini, DeepSeek, Groq)"""
    response = client.chat.completions.create(model=model_name, messages=messages)
    return response.choices[0].message.content

def call_anthropic(client, messages, model_name: str) -> str:
    """For Anthropic's API"""
    response = client.messages.create(model=model_name, messages=messages, max_tokens=1000)
    return response.content[0].text

# 1. The wrapper: It must take the function AND its arguments separately
async def run_model_async(name, func, *args):
    result = await asyncio.to_thread(func, *args)
    return {"name": name, "result": result}
    
# 2. Create a list of tasks (coroutines)
# Notice we are NOT using parentheses () after call_openai_compatible
# We pass the function object and the arguments as separate parameters
tasks = [
    run_model_async("openai", call_openai_compatible, openai_client, messages, openai_model),
    run_model_async("anthropic", call_anthropic, claude_client, messages, claude_model),
    run_model_async("gemini", call_openai_compatible, gemini_client, messages, gemini_model),
    run_model_async("deepseek", call_openai_compatible, deepseek_client, messages, deepseek_model),
    run_model_async("groq", call_openai_compatible, groq_client, messages, groq_model),
    run_model_async("ollama", call_openai_compatible, ollama_client, messages, ollama_model)
]
    
# asyncio.gather runs them all at once and waits for completion
# The '*' unpacks the list of tasks into arguments for gather
results_with_names = await asyncio.gather(*tasks)

print(results_with_names)

# This is your 'results_with_no_name'
results_with_no_name = [{"result": item["result"]} for item in results_with_names]

[{'name': 'openai', 'result': 'Here’s a concrete, non-linguistic, non-visual plan you could use to convince a skeptical alien that you understand the concept of red. The core idea is to ground the concept in objective physics (light’s spectrum) and in repeatable, verifiable behavior, and to show you can generalize and reason about it without using human language or pictures.\n\n1) Fix a universal, objective definition (in physics terms)\n- Use the spectral power distribution (SPD) of light as the input. An SPD is a function S(λ) giving power at each wavelength λ.\n- Define a formal, machine-verifiable criterion for “red”: there is a fixed long-wavelength emphasis rule. For example, choose a weighting function W(λ) that peaks around ~700 nm and define a red-score R = ∫ S(λ) W(λ) dλ.\n- Declare that a stimulus is labeled red if R exceeds a chosen threshold τ, and not-red otherwise. This is a mathematical, instrument-checked definition not dependent on language or pictures.\n- Note: brigh

In [10]:
together = ""
for index, item in enumerate(results_with_no_name):
    together += f"# Response from competitor {index+1}\n\n"
    together += item["result"] + "\n\n"
    
print(together)

# Response from competitor 1

Here’s a concrete, non-linguistic, non-visual plan you could use to convince a skeptical alien that you understand the concept of red. The core idea is to ground the concept in objective physics (light’s spectrum) and in repeatable, verifiable behavior, and to show you can generalize and reason about it without using human language or pictures.

1) Fix a universal, objective definition (in physics terms)
- Use the spectral power distribution (SPD) of light as the input. An SPD is a function S(λ) giving power at each wavelength λ.
- Define a formal, machine-verifiable criterion for “red”: there is a fixed long-wavelength emphasis rule. For example, choose a weighting function W(λ) that peaks around ~700 nm and define a red-score R = ∫ S(λ) W(λ) dλ.
- Declare that a stimulus is labeled red if R exceeds a chosen threshold τ, and not-red otherwise. This is a mathematical, instrument-checked definition not dependent on language or pictures.
- Note: brightness (

In [11]:
judge = f"""You are judging a competition between {len(results_with_no_name)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


judge_messages = [{"role": "user", "content": judge}]


In [12]:
# Judgement time!

# openai = OpenAI()
judgment_result = call_openai_compatible(openai, judge_messages, 'gpt-5-mini')

print(judgment_result)


{"results": ["5", "1", "3", "2", "4", "6"]}


In [13]:
# OK let's turn this into results!
judgement_result_dict = json.loads(judgment_result)
ranks = judgement_result_dict["results"]
print(ranks)

for index, item in enumerate(ranks):
    competitor = results_with_names[int(item)-1]["name"]
    print(f"Rank {index + 1}: {competitor}")

['5', '1', '3', '2', '4', '6']
Rank 1: groq
Rank 2: openai
Rank 3: gemini
Rank 4: anthropic
Rank 5: deepseek
Rank 6: ollama
